In [68]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1337);

In [69]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-11-21 15:38:33--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1,1M) [text/plain]
Saving to: 'input.txt.1'

input.txt.1         100%[===================>]   1,06M  1,02MB/s    in 1,0s    

2023-11-21 15:38:35 (1,02 MB/s) - 'input.txt.1' saved [1115394/1115394]



In [70]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [71]:
len(text) # number of chars in the text

1115394

In [72]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [73]:
# create a vocab (all the unique chars in the text)
chars = sorted(set(text)) # list
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [74]:
stoi = {s: i for i, s in enumerate(chars)}
itos = {i: s for i, s in enumerate(chars)}
encode = lambda x: [stoi[c] for c in x] # encoder: take a string, output a list of ints
decode = lambda s: "".join([itos[c] for c in s]) # decoder: take a list of ints, output a string

print(encode("siema"))
print(decode(encode("siema")))

[57, 47, 43, 51, 39]
siema


In [75]:
# let's now encode our text and store it in a vector
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [76]:
# train/val split
train_n = int(0.9*len(text))
train_data = data[:train_n]
val_data = data[train_n:]
print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [77]:
# divide into smaller CHUNKS (less computationally intensive than feeding everything at once)
context_len = 8 # block_size

In [78]:
x = train_data[:context_len]
y = train_data[1:context_len+1]

for i in range(context_len):
    context = x[:i+1]
    target = y[i]
    print(f"when input is {context}, the target is: {target}")

when input is tensor([18]), the target is: 47
when input is tensor([18, 47]), the target is: 56
when input is tensor([18, 47, 56]), the target is: 57
when input is tensor([18, 47, 56, 57]), the target is: 58
when input is tensor([18, 47, 56, 57, 58]), the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [79]:
# divide into batches so that many chunks can be passed to a transformer at once (for efficiency)
batch_size = 4 # how many independent examples will we process in parallel?

def get_batch(split):
    data = train_data if split == "train" else val_data
    idx = torch.randint(len(data) - context_len, (batch_size,)) # batch_size numbers between 0 and len(data) - context_len
    x = torch.stack([data[i:i+context_len] for i in idx])
    y = torch.stack([data[i+1:i+context_len+1] for i in idx])
    return x, y

In [80]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(context_len): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [128]:
# most basic model - bigram
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size): # "__init__() call to the parent class must be made before assignment on the child."
        super().__init__()
        
        # 65 x 65 table
        # https://stackoverflow.com/questions/50747947/embedding-in-pytorch
        # "nn.Embedding holds a Tensor of dimension (vocab_size, vector_size), 
        # i.e. of the size of the vocabulary x the dimension of each vector embedding, and a method that does the lookup.
        # When you create an embedding layer, the Tensor is initialised randomly. 
        # It is only when you train it when this similarity between similar words should appear."
        
        # each token directly reads off the logits for the next token from a lookup table
        # these just become probabilities for all the 65 chars to be the next char, given some char
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # "A simple lookup table that stores embeddings of a fixed dictionary and size."
    
    def forward(self, idx, targets=None): # "Defines the computation performed at every call. Should be overridden by all subclasses."
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss 

    def generate(self, idx, max_new_tokens):
        # now it's pretty bad, because we only use the last char but later it will change!
        for _ in range(max_new_tokens):
            logits, _ = self(idx) # get predictions (these embeddings) 
            logits = logits[:, -1, :] # get predictions only for the next character (thus -1) -> (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            # plucks out index of the char (0-64) with the highest prob. index in the probs tensor corresponds to actual index in the emb table:
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) -> "If input is a matrix with m rows, out is an matrix of shape (m×num_samples)."
            # append sampled index to the running sequence:
            idx = torch.cat((idx, idx_next), dim=1) # dim=1 -> concatenate "Time" -> (B, C) becomes (B, C+1)
        return idx

In [129]:
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb) # spits out a row from the embedding table from each example from xb in a batch
print(logits, logits.shape, loss) # -ln(1/65) ~= 4.1743

tensor([[ 0.1096, -1.4002, -2.0709,  ..., -0.0301, -2.6801, -0.1619],
        [ 0.0362,  1.5888, -1.5286,  ...,  0.7430,  1.1266, -0.1201],
        [-0.2159,  0.4251, -0.2401,  ..., -0.3643,  1.0510,  1.2662],
        ...,
        [-0.1066,  1.5575,  1.3555,  ...,  0.1655, -1.3049,  0.6517],
        [ 0.0069, -0.2607, -0.9520,  ...,  0.0702, -0.4472, -0.2668],
        [-0.2970,  0.2681,  0.6003,  ..., -0.1294, -0.5774, -1.1707]],
       grad_fn=<ViewBackward0>) torch.Size([32, 65]) tensor(4.6029, grad_fn=<NllLossBackward0>)


In [143]:
init_idx = torch.zeros((1, 1), dtype=torch.long) # B=1, T=1 and it's holding a 0 (0 is a new line so a reasonable char to start with)
# encoded = model.generate(init_idx, max_new_tokens=100) -> print to see why line above is correct and this is not
encoded = model.generate(init_idx, max_new_tokens=200)[0].tolist() # [0] -> 0th batch dimension
decoded = decode(encoded)
print(decoded) # terrible



Tor ry waldisthee f abe henend thine earngad pld okneth theather MENTIFord iveho ce hipurtasthulf t routorern ngbaccriseerrd s very qut, o thisthilerdilindgr for thain.

Heyo ofue, sen h ditoreaillla


In [137]:
# FIND OUT WHAT'S THE DIFFERENCE BETWEEN SGD AND THAT!!!
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [136]:
batch_size = 32
for steps in range(100000): # 100000 -> loss: 2.3590312004089355
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward() # get gradients for all of the parameters
    optimizer.step() # update the parameters according to the gradient

print(loss.item())

2.3590312004089355


In [147]:
init_idx = torch.zeros((1, 1), dtype=torch.long) # B=1, T=1 and it's holding a 0 (0 is a new line so a reasonable char to start with)
# encoded = model.generate(init_idx, max_new_tokens=100) -> print to see why line above is correct and this is not
encoded = model.generate(init_idx, max_new_tokens=200)[0].tolist() # [0] -> 0th batch dimension
decoded = decode(encoded)
print(decoded) # still really bad, but at least there is some structure now


Ana wigomy ngrathe w wateanevearrotronon'se wal y athan!


Shel d
Th e hieepo l'd d gid aimenat n foser

Weplisthyo. brin; VI goowat ber bl sor higot s, Shair nd ather p ve he
Forave in ck who. n hond
